In [5]:
import cv2
import numpy as np
from paddleocr import PaddleOCR
from PIL import Image, ImageDraw, ImageFont

# Load the image and get its dimensions
image_path = "../Death Certificates/Death_Certificate.jpg"
img_cv = cv2.imread(image_path)
h, w, _ = img_cv.shape

# Create a blank white image using PIL with the same dimensions
blank_image = Image.new('RGB', (w, h), (255, 255, 255))
draw = ImageDraw.Draw(blank_image)

# Initialize PaddleOCR (CPU mode)
ocr = PaddleOCR(use_angle_cls=True, lang="en")

# Run OCR on the image
result = ocr.ocr(image_path, cls=True)

# Function to determine the best font size
def get_best_fit_font(text, bbox_width, bbox_height):
    """ Dynamically find the best font size that fits inside the bounding box """
    font_size = bbox_height  # Start with bbox height
    while font_size > 5:  # Ensure font is not too small
        try:
            font = ImageFont.truetype("arial.ttf", font_size)
        except:
            font = ImageFont.load_default()
        
        text_width, text_height = draw.textbbox((0, 0), text, font=font)[2:]
        if text_width <= bbox_width and text_height <= bbox_height:
            return font
        font_size -= 1  # Reduce size if it doesn't fit
    return font  # Return the smallest possible fitting font

# Loop over each detected text block and draw the text at its bounding box's exact position
for line in result:
    for word in line:
        text = word[1][0]  # Extract detected text
        bbox = np.array(word[0])  # Bounding box coordinates (4 points)

        # Get bounding box position
        x1, y1 = bbox[0]  # Top-left
        x2, y2 = bbox[2]  # Bottom-right

        bbox_width = x2 - x1
        bbox_height = y2 - y1

        # Get the best font size to match the text dimensions
        font = get_best_fit_font(text, bbox_width, bbox_height)

        # Compute the angle of rotation
        dx, dy = bbox[1] - bbox[0]  # Vector along the top of the bounding box
        angle = np.degrees(np.arctan2(dy, dx))  # Compute angle in degrees

        # Create a temporary transparent image for rotated text
        text_layer = Image.new("RGBA", (w, h), (255, 255, 255, 0))
        text_draw = ImageDraw.Draw(text_layer)

        # Draw text at (x1, y1)
        text_draw.text((x1, y1), text, fill=(0, 0, 0, 255), font=font)

        # Rotate the text layer to match the detected text angle
        rotated_text_layer = text_layer.rotate(angle, center=(x1, y1), resample=Image.BICUBIC)

        # Paste rotated text onto the blank image
        blank_image.paste(rotated_text_layer, mask=rotated_text_layer)

# Display the resulting image
blank_image.show()

# Save the final output
blank_image.save("text_recreated.jpg")

[2025/02/20 18:51:36] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\LogaSanjeev/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\LogaSanjeev/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', re